#### Notebook to save input and output data to load in the future

In [2]:
import torch
from Bio import SeqIO
import pyBigWig
from helpers import seq_to_one_hot

#### Currently, I subsampled the data by getting 10.000 samples from each chromosome.

In [2]:
start_index = 10_000_000
end_index = 20_000_000
sequence_length = 1000
num_samples = (end_index - start_index) // sequence_length
num_tasks = 4
tasks = ['oct4', 'sox2', 'nanog', 'klf4']

chromosomes = ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
                'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr3', 'chr4',
                'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrX', 'chrY']

train_chrs = ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
                'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9']

val_chrs = ['chr16', 'chr17', 'chr18']
test_chrs = ['chr19', 'chrX', 'chrY']

In [3]:
dna_file = 'mm10.fa'
with open(dna_file, 'r') as file:
        dna = list(SeqIO.parse(file, 'fasta'))

In [4]:
# get labels
pos_counts = []
neg_counts = []
for task in tasks:
    pos_counts.append(pyBigWig.open(f'{task}-counts.pos.bw'))
    neg_counts.append(pyBigWig.open(f'{task}-counts.neg.bw'))

In [5]:
train_sequences = list(filter(lambda c: c.id in train_chrs, dna))
val_sequences = list(filter(lambda c: c.id in val_chrs, dna))
test_sequences = list(filter(lambda c: c.id in test_chrs, dna))

In [ ]:
datasets = {
    'train': train_sequences,
    'val': val_sequences,
    'test': test_sequences
}

for dataset_name, dataset in datasets.items():
    inputs = torch.zeros(len(dataset) * num_samples, sequence_length, 4)
    targets_chip_seq = torch.zeros(len(dataset) * num_samples, num_tasks, sequence_length, 2)
    targets_bias = torch.zeros(len(dataset) * num_samples, num_tasks, 2)
    for chr in range(len(dataset)):
        name_chr = dataset[chr].id
        print(dataset[chr].id)
        for i in range(start_index, end_index, sequence_length):
            start = i
            end = i + sequence_length
            index = chr * num_samples + (i - start_index) // 1000
            inputs[index] = seq_to_one_hot(str(dataset[chr].seq[start:end]))
            for task in range(num_tasks):
                # get profile shape for both positive and negative strands
                ps_pos = torch.Tensor(pos_counts[task].values(name_chr, start, end))
                ps_neg = torch.Tensor(neg_counts[task].values(name_chr, start, end))
                ps_pos[ps_pos != ps_pos] = 0
                ps_neg[ps_neg != ps_neg] = 0
                # get total counts
                tc_pos = ps_pos.sum()
                tc_neg = ps_neg.sum()
                targets_chip_seq[index][task] = torch.stack([ps_pos, ps_neg], dim=1)
                targets_bias[index][task] = torch.Tensor([tc_pos, tc_neg])

    torch.save(inputs, f'{dataset_name}_inputs.pt')
    torch.save(targets_chip_seq, f'{dataset_name}_targets_chip_seq.pt')
    torch.save(targets_bias, f'{dataset_name}_targets_bias.pt')
